## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,84,59,9.19,broken clouds
1,1,Jalu,LY,29.0331,21.5482,79.14,49,54,14.99,broken clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,32.00,74,40,9.22,scattered clouds
3,3,Hilo,US,19.7297,-155.0900,77.79,91,90,0.00,moderate rain
4,4,Ushuaia,AR,-54.8000,-68.3000,46.02,49,75,14.97,broken clouds
5,5,Bundaberg,AU,-24.8500,152.3500,64.22,93,100,4.00,overcast clouds
6,6,Kudahuvadhoo,MV,2.6708,72.8944,82.18,70,78,6.69,broken clouds
7,7,Ponta Do Sol,PT,32.6667,-17.1000,73.71,74,5,9.13,clear sky
8,8,Mar Del Plata,AR,-38.0023,-57.5575,59.02,31,75,11.99,broken clouds
9,9,Tumpat,MY,6.1978,102.1710,77.02,88,20,3.44,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,84,59,9.19,broken clouds
1,1,Jalu,LY,29.0331,21.5482,79.14,49,54,14.99,broken clouds
3,3,Hilo,US,19.7297,-155.0900,77.79,91,90,0.00,moderate rain
6,6,Kudahuvadhoo,MV,2.6708,72.8944,82.18,70,78,6.69,broken clouds
7,7,Ponta Do Sol,PT,32.6667,-17.1000,73.71,74,5,9.13,clear sky
9,9,Tumpat,MY,6.1978,102.1710,77.02,88,20,3.44,few clouds
13,13,Faanui,PF,-16.4833,-151.7500,79.21,72,1,17.74,clear sky
14,14,Georgetown,MY,5.4112,100.3354,82.33,91,40,5.99,scattered clouds
20,20,Puri,IN,19.8000,85.8500,81.12,80,73,8.48,broken clouds
29,29,Castro,BR,-24.7911,-50.0119,76.03,51,39,9.86,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                301
City                   301
Country                300
Lat                    301
Lng                    301
Max Temp               301
Humidity               301
Cloudiness             301
Wind Speed             301
Current Description    301
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                300
City                   300
Country                300
Lat                    300
Lng                    300
Max Temp               300
Humidity               300
Cloudiness             300
Wind Speed             300
Current Description    300
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
1,Jalu,LY,79.14,broken clouds,29.0331,21.5482,
3,Hilo,US,77.79,moderate rain,19.7297,-155.0900,
6,Kudahuvadhoo,MV,82.18,broken clouds,2.6708,72.8944,
7,Ponta Do Sol,PT,73.71,clear sky,32.6667,-17.1000,
9,Tumpat,MY,77.02,few clouds,6.1978,102.1710,
13,Faanui,PF,79.21,clear sky,-16.4833,-151.7500,
14,Georgetown,MY,82.33,scattered clouds,5.4112,100.3354,
20,Puri,IN,81.12,broken clouds,19.8000,85.8500,
29,Castro,BR,76.03,scattered clouds,-24.7911,-50.0119,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,Jalu,LY,79.14,broken clouds,29.0331,21.5482,صاحبة الفخامة
3,Hilo,US,77.79,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Kudahuvadhoo,MV,82.18,broken clouds,2.6708,72.8944,Niyama Private Islands Maldives
7,Ponta Do Sol,PT,73.71,clear sky,32.6667,-17.1000,Hotel do Campo
9,Tumpat,MY,77.02,few clouds,6.1978,102.1710,Baan Tanyong B&B
13,Faanui,PF,79.21,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
14,Georgetown,MY,82.33,scattered clouds,5.4112,100.3354,Cititel Penang
20,Puri,IN,81.12,broken clouds,19.8000,85.8500,OYO 1577 Hotel Shree Hari Grand
29,Castro,BR,76.03,scattered clouds,-24.7911,-50.0119,CHACARA BAILLY


In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))